In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Преобразования для данных
transform = transforms.Compose([
    transforms.ToTensor(),  # Преобразуем изображения в тензоры
    transforms.Normalize((0.5,), (0.5,))  # Нормализация
])

# Загрузка тренировочного и тестового наборов данных
train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

# Создание DataLoader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

100%|██████████| 26.4M/26.4M [00:02<00:00, 12.7MB/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 210kB/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:01<00:00, 3.92MB/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 5.18MB/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [2]:
class FashionMNISTCNN(nn.Module):
    def __init__(self):
        super(FashionMNISTCNN, self).__init__()
        # Первый свёрточный слой
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Второй свёрточный слой
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Полносвязный слой
        self.fc1 = nn.Linear(64 * 7 * 7, 128)  # 7x7 - размер изображения после пулинга
        self.relu3 = nn.ReLU()
        self.dropout = nn.Dropout(0.5)  # Регуляризация
        self.fc2 = nn.Linear(128, 10)  # 10 классов в Fashion MNIST

    def forward(self, x):
        # Первый свёрточный блок
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)

        # Второй свёрточный блок
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)

        # Выравнивание для полносвязного слоя
        x = x.view(x.size(0), -1)

        # Полносвязный слой
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.dropout(x)
        x = self.fc2(x)

        return x

# Инициализация модели
model = FashionMNISTCNN()

In [3]:
# Функция потерь
criterion = nn.CrossEntropyLoss()

# Оптимизатор
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [4]:
# Функция для вычисления точности
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

# Обучение модели
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    # Тренировка
    model.train()
    train_loss = 0.0
    train_acc = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Обнуление градиентов
        optimizer.zero_grad()

        # Прямой проход
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Обратный проход и оптимизация
        loss.backward()
        optimizer.step()

        # Суммирование потерь и точности
        train_loss += loss.item()
        train_acc += accuracy(outputs, labels)

    # Вычисление средних потерь и точности
    train_loss /= len(train_loader)
    train_acc /= len(train_loader)

    # Тестирование
    model.eval()
    test_loss = 0.0
    test_acc = 0.0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            test_acc += accuracy(outputs, labels)

    # Вычисление средних потерь и точности
    test_loss /= len(test_loader)
    test_acc /= len(test_loader)

    # Вывод результатов
    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
          f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")

    # Если точность на тестовом наборе достигла 89,5%, останавливаем обучение
    if test_acc >= 0.895:
        print("Требуемая точность достигнута!")
        break

Epoch [1/10], Train Loss: 0.5706, Train Acc: 0.7942, Test Loss: 0.3620, Test Acc: 0.8674
Epoch [2/10], Train Loss: 0.3763, Train Acc: 0.8654, Test Loss: 0.2986, Test Acc: 0.8882
Epoch [3/10], Train Loss: 0.3193, Train Acc: 0.8858, Test Loss: 0.2689, Test Acc: 0.9017
Требуемая точность достигнута!
